In [1]:
import json
import os
import openai
# from dotenv import load_dotenv
# import anthropic

In [ ]:
# # Read QA questions from the json file
# with open('shuffled_QA_pairs_econ_part1.json') as f:
#     data1 = json.load(f)

# with open('shuffled_QA_pairs_econ_part2.json') as f:
#     data2 = json.load(f)

# data_merged = data1 + data2
# question_set = set()
# data = []
# for row in data_merged:
#     if row['question'] not in question_set:
#         question_set.add(row['question'])
#         data.append(row)
#     else:
#         continue

# with open('shuffled_QA_pairs_econ.json', 'w') as f:
#     json.dump(data, f)

with open('shuffled_QA_pairs_econ.json') as f:
    data = json.load(f)
print(len(data))
# Load the .env file
# load_dotenv()
# client = openai.OpenAI()
# claude_client = anthropic.Anthropic()
client = openai.AzureOpenAI(api_version='2024-06-01')


In [ ]:
from tqdm import tqdm
from collections import defaultdict
log = defaultdict(dict)


for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['mcq']

    prompt = f"""
    Question: {choices}
    Which of the above choices best answers the question? Give a reason for your choice first, then give your choice. Think step by step.
    Reply a JSON object with the following format:
    {{
        "reason": "string",
        "choice": "char",
    }}
    """
    temperature = 0.0

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a student answering a textbook problem."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    
    try:
        answer_json = json.loads(response.choices[0].message.content)
        log[question]['reason'] = answer_json['reason']
        log[question]['gpt_choice'] = answer_json['choice']
        log[question]['correct_choice'] = correct_answer + 1
    except:
        print(response.choices[0].message.content)
        log[question]['reason'] = ""
        log[question]['gpt_choice'] = ""
        log[question]['correct_choice'] = correct_answer + 1

    char2num_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
    log[question]['incorrect_choices'] = [char2num_map[char.upper()] for char in log[question]['incorrect_choices']]

In [ ]:
correct_count = 0
total_count = len(log)

for _, answers in log.items():
    if answers['gpt_choice'] == answers['correct_choice']:
        correct_count += 1

percentage_correct = (correct_count / total_count) * 100
print(f"Percentage of correct choices: {percentage_correct:.2f}%")

In [6]:
import pickle
with open('log_all_cot.pkl', 'wb') as f:
    pickle.dump(log, f)

### Select all wrong answers

In [ ]:
with open('shuffled_QA_pairs_econ.json') as f:
    data = json.load(f)
print(len(data))
client = openai.AzureOpenAI(api_version='2024-06-01')

from tqdm import tqdm
from collections import defaultdict
log = defaultdict(dict)


for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['mcq']

    prompt = f"""
    Question: {choices}
    Select all wrong choices. Give reasons for your choices, then give your choices. Think step by step.
    Reply a JSON object with the following format:
    {{
        "analysis": {{
            "choice_A": "Reason why choice A is correct/incorrect",
            "choice_B": "Reason why choice B is correct/incorrect",
            "...": "..."
        }},
        "incorrect_choices": [char, char, ...]
    }}
    """
    print(prompt)
    temperature = 0.0

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a student answering a textbook problem."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    
    try:
        answer_json = json.loads(response.choices[0].message.content)
        log[question]['analysis'] = answer_json['analysis']
        log[question]['incorrect_choices'] = answer_json['incorrect_choices']
        log[question]['correct_choice'] = correct_answer + 1
    except:
        print(response.choices[0].message.content)
        log[question]['analysis'] = ""
        log[question]['incorrect_choices'] = []
        log[question]['correct_choice'] = correct_answer + 1

    char2num_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
    log[question]['incorrect_choices'] = [char2num_map[char.upper()] for char in log[question]['incorrect_choices']]


In [5]:
import pickle
with open('log_pick_wrong.pkl', 'wb') as f:
    pickle.dump(log, f)

### Select all correct answers

In [2]:
with open('shuffled_QA_pairs_econ.json') as f:
    data = json.load(f)
print(len(data))
client = openai.AzureOpenAI(api_version='2024-06-01')

from tqdm import tqdm
from collections import defaultdict
log = defaultdict(dict)


for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['mcq']

    prompt = f"""
    Question: {choices}
    Select all correct choices. Give reasons for your choices, then give your choices. Think step by step.
    Reply a JSON object with the following format:
    {{
        "analysis": {{
            "choice_A": "Reason why choice A is correct/incorrect",
            "choice_B": "Reason why choice B is correct/incorrect",
            "...": "..."
        }},
        "correct_choices": [char, char, ...]
    }}
    """
    temperature = 0.0

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a student answering a textbook problem."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    
    try:
        answer_json = json.loads(response.choices[0].message.content)
        log[question]['analysis'] = answer_json['analysis']
        log[question]['correct_choices'] = answer_json['correct_choices']
        log[question]['correct_choice'] = correct_answer + 1
    except:
        print(response.choices[0].message.content)
        log[question]['analysis'] = ""
        log[question]['correct_choices'] = []
        log[question]['correct_choice'] = correct_answer + 1

    char2num_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
    log[question]['correct_choices'] = [char2num_map[char.upper()] for char in log[question]['correct_choices']]


338


Generating answers: 100%|██████████| 338/338 [28:15<00:00,  5.02s/it] 


In [ ]:
import pickle
with open('log_pick_correct.pkl', 'rb') as f:
    log = pickle.load(f)

In [ ]:
log[list(log.keys())[0]]


In [14]:
# Read QA questions from the json file
with open('shuffled_QA_pairs_econ.json') as f:
    data = json.load(f)

# Load the .env file
# load_dotenv()
# client = openai.OpenAI()
# claude_client = anthropic.Anthropic()
client = openai.AzureOpenAI(api_version='2024-06-01')


In [15]:
import re

for i, q in enumerate(data):    
    # remove the empty newlines and first few numbers followed by a period in the question by regex    
    clean_question = re.sub(r'^\s*\d+\.\s*', '', q['question'])    
    data[i]['clean_question'] = clean_question

In [ ]:
from tqdm import tqdm
from collections import defaultdict
classification_log = defaultdict(dict)

for generated_qa in tqdm(data, desc="Generating answers"):
    question_key = generated_qa['question']
    question = generated_qa['clean_question']

    system_prompt = """
    You will be presented with a problem pulled from a finance textbook. 
    Your goal is to determine three things: whether the question has enough information to be answered, if it does, what type of question it is, and if it is fit for use as a multiple-choice question.
    
    When determining if the question has enough information, classify it as one of the following:
    1. Incomplete: Is the question complete? Are there any missing words or phrases, or references to other parts of the text that are not included?
    2. Missing reference: Does the question contain all the reference materials? Are there any missing tables, charts, or other information that is necessary to answer the question?
    3. Missing context: Does the question contain all the necessary context? Does it ask for information from sections in the textbook, which are not included in the question?
    9. Other: The question does not have enough information, but it does not fit into any of the above categories.
    0. Complete: The question has all the necessary information to be answered.

    When determining the type of question, classify the question as one of the following:
    1. Conceptual: The question asks for an explanation of a concept or theory.
    2. Computation: The question asks for a calculation or numerical answer.
    3. True/False: The question asks for a true or false answer.
    4. Graphical: The question asks for a graph or chart to be drawn.
    5. Reasoning: The question asks for a logical explanation or reasoning.
    9. Other: The question does not fit into any of the above categories.

    When determining if the question is fit for use as a multiple-choice question, simply answer yes or no by assigning a 1 or 0, respectively.

    You will also be asked to provide a reason for your answer. Please provide a detailed explanation for your choice.
    When you answer, use the associated number to indicate your choice for each of the three questions.
    For example, a complete question that is a conceptual question and is not fit for use as a multiple-choice question would be answered with:
    0, 1, 0
    """

    question_prompt = f"""
    Question: {question}
    For your answer, reply a JSON object with the following format:
    {{
        "completeness_reason": "string",
        "type_reason": "string",
        "mcq_reason": "string",
        "completeness_choice": "int",
        "type_choice": "int",
        "mcq_choice": "int"
    }}
    """
    temperature = 0.0

    response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": question_prompt}
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
    
    answer_json = json.loads(response.choices[0].message.content)

    answer_dict = {
        "completeness_reason": answer_json['completeness_reason'],
        "type_reason": answer_json['type_reason'],
        "mcq_reason": answer_json['mcq_reason'],
        "completeness_choice": answer_json['completeness_choice'],
        "type_choice": answer_json['type_choice'],
        "mcq_choice": answer_json['mcq_choice']
    }

    classification_log[question_key] = answer_dict

In [20]:
import pickle
with open('classification_log_all.pkl', 'wb') as f:
    pickle.dump(classification_log, f)

In [11]:
import pickle
with open('log.pkl', 'rb') as f:
    log = pickle.load(f)

In [12]:
import pickle
with open('classification_log.pkl', 'rb') as f:
    classification_log = pickle.load(f)

In [ ]:
classification_log[list(classification_log.keys())[0]]


In [12]:
for original_qa in data:
    question = original_qa['question']
    log[question]['question'] = original_qa['mcq']

In [14]:
for i, individual_data in enumerate(data):
    question = individual_data['question']
    log[question]['index'] = i

In [ ]:
import random

for _ in range(10):
    question = random.choice(data)['question']
    print(f"Question: {log[question]['question']}")
    print(f"Correct choice: {log[question]['correct_choice'] + 1}")
    print(f"GPT choice: {log[question]['gpt_choice']}")
    print(f"Reason: {log[question]['reason']}")
    print()
    print()

In [ ]:
from anthropic.types.beta.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.beta.messages.batch_create_params import Request
requests = []

for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['mcq']

    prompt = f"""
    Question: {choices}
    Which of the above choices best answers the question? Give a reason for your choice first, then give your choice as an integer between 0 and 3.
    Reply only a JSON object with the following format:
    {{
        "reason": "string",
        "choice": "int",
    }}
    """
    temperature = 0.0
    params = MessageCreateParamsNonStreaming(
                model="claude-3-5-haiku-latest",
                system = "You are a student answering a textbook problem.",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=1000,
            )
    
    custom_id = str(hash(question))
    requests.append(Request(custom_id=custom_id, params=params))
    break
    
message_batch = claude_client.beta.messages.batches.create(requests=requests)

In [ ]:
anthropic_log = defaultdict(dict)

batch_id = message_batch.id

for result in claude_client.beta.messages.batches.results(batch_id):
    if result.result.type == "succeeded":
        print(f"Success! {result.custom_id}")


# answer_json = json.loads(response.content[0].text)

#     anthropic_log[question]['reason'] = answer_json['reason']
#     anthropic_log[question]['gpt_choice'] = answer_json['choice']
#     anthropic_log[question]['correct_choice'] = correct_answer

In [ ]:
correct_count = 0
total_count = len(anthropic_log)

for _, answers in log.items():
    if answers['gpt_choice'] == answers['correct_choice'] + 1:
        correct_count += 1

percentage_correct = (correct_count / total_count) * 100
print(f"Percentage of correct choices: {percentage_correct:.2f}%")